# Notebook 1: EDA

In [66]:
import kaggle
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [67]:
# Use Kaggle API to automatically download dataset.
# obs: more info in https://www.kaggle.com/docs/api
os.system('kaggle datasets download -d uciml/default-of-credit-card-clients-dataset -p ./data --unzip')

0

## Extract

In [68]:
df = pd.read_csv('data/UCI_Credit_card.csv')
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## Data Quality

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [70]:
# Missing values?
df.loc[df.isna().any(axis=1)]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month


In [71]:
# Duplicates?
df.loc[df.duplicated()]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month


In [72]:
# Unique ID?
df['ID'].value_counts(ascending=False).head()

ID
1        1
19997    1
20009    1
20008    1
20007    1
Name: count, dtype: int64

In [73]:
# LIMIT_BAL > 0?
# If float do not have decimals, it will be turned into int.
def remove_decimals(value):
    if value.is_integer():
        return int(value)
    else:
        return value

df['LIMIT_BAL'] = df['LIMIT_BAL'].apply(remove_decimals)
df['LIMIT_BAL'].describe()

count      30000.000000
mean      167484.322667
std       129747.661567
min        10000.000000
25%        50000.000000
50%       140000.000000
75%       240000.000000
max      1000000.000000
Name: LIMIT_BAL, dtype: float64

In [74]:
# Binary default.payment.next.month?
df['default.payment.next.month'].value_counts(dropna=False)

default.payment.next.month
0    23364
1     6636
Name: count, dtype: int64

In [79]:
# Binary SEX?
df['SEX'].value_counts(dropna=False)

SEX
2    18112
1    11888
Name: count, dtype: int64

In [80]:
# Fixed categories for EDUCATION?
df['EDUCATION'].value_counts(dropna=False)

EDUCATION
2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: count, dtype: int64

In [81]:
# Fixed categories for MARRIAGE?
df['MARRIAGE'].value_counts(dropna=False)

MARRIAGE
2    15964
1    13659
3      323
0       54
Name: count, dtype: int64

In [82]:
# 0 < AGE < 200?
df['AGE'].describe()

count    30000.000000
mean        35.485500
std          9.217904
min         21.000000
25%         28.000000
50%         34.000000
75%         41.000000
max         79.000000
Name: AGE, dtype: float64

In [101]:
# PAY_0 to PAY_6 between -1 and 1 to 9?
list_pay = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
pd.unique(df[list_pay].values.ravel())

array([ 2, -1, -2,  0,  1,  3,  4,  5,  6,  8,  7], dtype=int64)

In [108]:
# BILL_AMT is between 0 and +inf?
list_bill = [i for i in list(df.columns) if 'BILL_AMT' in i]
df[list_bill].describe().round(1)

,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6
count,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0
mean,51223.3,49179.1,47013.2,43262.9,40311.4,38871.8
std,73635.9,71173.8,69349.4,64332.9,60797.2,59554.1
min,-165580.0,-69777.0,-157264.0,-170000.0,-81334.0,-339603.0
25%,3558.8,2984.8,2666.2,2326.8,1763.0,1256.0
50%,22381.5,21200.0,20088.5,19052.0,18104.5,17071.0
75%,67091.0,64006.2,60164.8,54506.0,50190.5,49198.2
max,964511.0,983931.0,1664089.0,891586.0,927171.0,961664.0


In [109]:
# positive PAY_AMT?
list_pay_amt = [i for i in list(df.columns) if 'PAY_AMT' in i]
df[list_pay_amt].describe().round(1)

,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
count,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0
mean,5663.6,5921.2,5225.7,4826.1,4799.4,5215.5
std,16563.3,23040.9,17607.0,15666.2,15278.3,17777.5
min,0.0,0.0,0.0,0.0,0.0,0.0
25%,1000.0,833.0,390.0,296.0,252.5,117.8
50%,2100.0,2009.0,1800.0,1500.0,1500.0,1500.0
75%,5006.0,5000.0,4505.0,4013.2,4031.5,4000.0
max,873552.0,1684259.0,896040.0,621000.0,426529.0,528666.0


### Results

Interpretation
- ✔️: OK
- ❌: problems

General checklist:
- missing values: ✔️
- datatype: ✔️
- duplicates: ✔️

Specific checklist:
- Is ID unique? ✔️
- Is LIMIT_BAL positive? ✔️
- Is default.payment.next.month binary? ✔️
- Is SEX binary? ✔️
- Is EDUCATION between 1 to 6? ❌
- Is MARRIAGE between 1 to 3? ❌
- Is AGE between 0 to 200? ✔️
- Is PAY between -1 or between 1 and 9? ❌
- Is BILL_AMT between -inf to +inf?* ✔️
- Is PAY_AMY positive? ✔️

*obs: negative billing values could mean refund.*



## Split train test dataset

In [75]:
# Split conditional do target.
X = df.drop(columns='default.payment.next.month')
y = df['default.payment.next.month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52, shuffle=True, stratify=y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(24000, 24) (6000, 24) (24000,) (6000,)


In [76]:
# Export.
X_train.to_csv('data/X_train.csv', index=False)
X_test.to_csv('data/X_test.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

## EDA

## Conclusion

### General

**Dataset description**: 30000 rows with no missing value.
- `ID`: unique, therefore, there are no multiple loans.
- `LIMIT_BAL`: total amount of credit limit in local currency.

**Data source**: according to the original author, the dataset was extract from a single relevant bank in October 2005, filtering only credit card holders of the bank. There is, however, a divergence between original paper's and UCI dataset's metadata. While the original paper states 25.000 rows and 5529 (4,52%) cases of credit default, the UCI dataset displays 30.000 rows and 6636 (4,52%), suggesting an increase in rows respecting the original default proportion of 4,52%. It is not clear how other variables were affected.

**Local context**: in March 2005, Mainland China passed a [Anti-Secession Law](!https://en.wikipedia.org/wiki/Anti-Secession_Law), increasing chinese political grasp into Taiwan. It is not clear how relevant the event was on the dataset or even if the effect was equally distributed over time. 

### Hypothesis


Ideas:

- if the bank have a specific economic class focus, it may have problems predicting credit default in other economic classes. For example, a bank specialized in low value loans may have a worse system for dealing with high value loans.

- 


